In [1]:
import mlflow

mlflow.__version__

'2.21.3'

In [2]:
from src.utils.folder_operations import get_project_root

# set mlflow tracking uri
mlflow.set_tracking_uri(
    # (get_project_root() / 'mlflow_new/mlruns').as_uri()
    "http://localhost:3000"
)

import os
os.environ['AWS_ACCESS_KEY_ID'] = 'mlflow'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'mlflow123'
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://localhost:9000'

# MLflow Model Registry

The MLflow Model Registry component is a centralized model store, set of APIs, and UI, to collaboratively manage the full lifecycle of an MLflow Model. It provides model lineage (which MLflow experiment and run produced the model), model versioning, model aliasing, model tagging, and annotations.

## Concepts

- Model
- Registered Model
- Model Version
- Model Alias
- Tags
- Annotations and descriptions

# Register a Model via UI

In [3]:
from sklearn.ensemble import RandomForestClassifier

In [4]:
client = mlflow.MlflowClient()

In [5]:
# instantiate a model
rfc = RandomForestClassifier()

# log the model
with mlflow.start_run(run_name="logging_model") as run:
    mlflow.sklearn.log_model(sk_model=rfc,artifact_path= rfc.__class__.__name__)

2025/05/07 11:30:54 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/05/07 11:30:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run logging_model at: http://localhost:3000/#/experiments/0/runs/4ce3cd327eb54eabb8dd4cdb715391cd
🧪 View experiment at: http://localhost:3000/#/experiments/0


# Registering a model using the Python SDK
Providing `registered_model_name` parameter

In [6]:
# instantiate a model
rfc = RandomForestClassifier(n_estimators=1)

with mlflow.start_run(run_name="registering_model_providing_name") as run:
    # log the model
    mlflow.sklearn.log_model(sk_model = rfc, artifact_path= rfc.__class__.__name__, registered_model_name="registered-model-sdk")

2025/05/07 11:31:50 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/05/07 11:31:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'registered-model-sdk'.
2025/05/07 11:31:50 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: registered-model-sdk, version 1


🏃 View run registering_model_providing_name at: http://localhost:3000/#/experiments/0/runs/67a9bf456c4b4d7e8a8277ea351ec6eb
🧪 View experiment at: http://localhost:3000/#/experiments/0


Created version '1' of model 'registered-model-sdk'.


In [7]:
run.info.artifact_uri

's3://mlflow/0/67a9bf456c4b4d7e8a8277ea351ec6eb/artifacts'

# Creating a registered model via SDK

## create_registered_model

In [8]:
model_name = "registered-model-via-client"

try:
    result = client.create_registered_model(name = model_name)
except Exception as e:
    # The method above fails if the model already exists
    print(e)

print(f"Model {result.name} created")
print(f"Model description: {result.description}")
print(f"Model creation timestamp: {result.creation_timestamp}")
print(f"Model Tags: {result.tags}")
print(f"Model Aliases: {result.aliases}")

Model registered-model-via-client created
Model description: 
Model creation timestamp: 1746628364348
Model Tags: {}
Model Aliases: {}


# Creating a model version

## create_model_version

In [9]:
try:
    model_version = client.create_model_version(
        name = model_name,
        source = run.info.artifact_uri,
        run_id = run.info.run_id,
        description = "Model Version created using MLflow Client")
except Exception as e:
    # The method above fails if the registered model does not exist.
    print(e)

print(f"Model version {model_version.version} created")
print(f"Model version status: {model_version.status}")
print(f"Model version description: {model_version.description}")
print(f"Model version creation timestamp: {model_version.creation_timestamp}")
print(f"Model version source: {model_version.source}")
print(f"Model version run_id: {model_version.run_id}")
print(f"Model version status_message: {model_version.status_message}")

2025/05/07 11:33:40 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: registered-model-via-client, version 1


Model version 1 created
Model version status: READY
Model version description: Model Version created using MLflow Client
Model version creation timestamp: 1746628420279
Model version source: s3://mlflow/0/67a9bf456c4b4d7e8a8277ea351ec6eb/artifacts
Model version run_id: 67a9bf456c4b4d7e8a8277ea351ec6eb
Model version status_message: None
